In [ ]:
from datetime import datetime, timedelta
from pyspark import SparkContext, SQLContext
from pyspark.sql.functions import hour, mean
import pyspark
sc = SparkContext.getOrCreate()

In [ ]:
spark = pyspark.sql.SparkSession.builder.appName("auctions").getOrCreate()
df_auctions = spark.read.csv('auctions.csv', header=True)
rdd_auctions = df_auctions.rdd
#AuctionTypeId, country, date, deviceId, platform, refType, sourceId

In [ ]:
spark = pyspark.sql.SparkSession.builder.appName("installs").getOrCreate()
df_installs = spark.read.csv('installs.csv', header=True)
rdd_installs = df_installs.rdd
#created, applicationId, refType, refHash, clickHash, attributed, implicit, deviceCountryCode, deviceBrand, deviceModel, sessionUserAgent, userAgent, eventUuid, kind, wifi, transId, ipAddress, deviceLanguage

In [ ]:
rdd_auctions = rdd_auctions.map(lambda x: (x.device_id, datetime.strptime(x.date, "%Y-%m-%d %H:%M:%S.%f")))
rdd_auctions = rdd_auctions.groupByKey().mapValues(list).mapValues(sorted)
rdd_installs = rdd_installs.map(lambda x: (x.device_id, datetime.strptime(x.created, "%Y-%m-%d %H:%M:%S.%f")))
rdd_installs = rdd_installs.groupByKey().mapValues(list).mapValues(sorted)

In [ ]:
auctionsMax = datetime.strptime("00:30:00", "%H:%M:%S")
installsMax = datetime.strptime("23:00:00", "%H:%M:%S")

In [ ]:
def get_timeDeltas_toMax_auctions(dateList):
    """
    esta funcion recibe una lista de fechas ordenada de las apariciones de un ID
    la idea es devolver una lista de tuplas, en la que cada tupla sea asi:
    valor 1 = ultima fecha en que aparecio en un auction
    valor 2 = distancia temporal hasta el siguiente pico de actividad (sea auctions o installs)
    """
    distancias = []
    longitud_actual = len(dateList)
    """
    En lastConversionIndexFound guardo el indice de la última conversión encontrada, 
    no quiero iterar n veces si aplica para otras fechas
    """
    date = dateList[longitud_actual - 1]
    timeDeltaToMax = (datetime(year = 1900, month = 1, day = 1, hour = date.hour, minute = date.minute, second = date.second) - auctionsMax).total_seconds()
    distancias.append(timeDeltaToMax)
        
    return distancias

In [ ]:
def get_timeDeltas_toMax_installs(dateList):
    """
    esta funcion recibe una lista de fechas ordenada de las apariciones de un ID
    la idea es devolver una lista de tuplas, en la que cada tupla sea asi:
    valor 1 = ultima fecha en que aparecio en un auction
    valor 2 = distancia temporal hasta el siguiente pico de actividad (sea auctions o installs)
    """
    distancias = []
    longitud_actual = len(dateList)
    """
    En lastConversionIndexFound guardo el indice de la última conversión encontrada, 
    no quiero iterar n veces si aplica para otras fechas
    """
    date = dateList[longitud_actual - 1]
    timeDeltaToMax = (datetime(year = 1900, month = 1, day = 1, hour = date.hour, minute = date.minute, second = date.second) - installsMax).total_seconds()
    distancias.append(timeDeltaToMax)
        
    return distancias

In [ ]:
rdd_auctions_topActivity_timeDeltas = rdd_auctions.flatMapValues(get_timeDeltas_toMax_auctions)
rdd_installs_topActivity_timeDeltas = rdd_installs.flatMapValues(get_timeDeltas_toMax_installs)

In [ ]:
def toCSVLine(data):
  return ','.join(str(d) for d in data)

In [ ]:
rdd_auctions_topActivity_timeDeltas.map(toCSVLine)
rdd_auctions_topActivity_timeDeltas.repartition(1).saveAsTextFile('auctions_topActivity.csv')

In [ ]:
rdd_installs_topActivity_timeDeltas.map(toCSVLine)
rdd_installs_topActivity_timeDeltas.repartition(1).saveAsTextFile('auctions_topActivity.csv')